In [ ]:
import overpy
import folium
from folium.plugins import HeatMap
import pandas as pd
import random

# Initialize Overpass API client
api = overpy.Overpass()

# Coordinates for Mansoura city and search radius (in meters)
latitude = 31.0348
longitude = 31.3539
radius = 10000

# Query to fetch restaurants, hospitals, and hotels
query = f"""
  [out:json];
  (
    node["amenity"="restaurant"](around:{radius},{latitude},{longitude});
    node["amenity"="hospital"](around:{radius},{latitude},{longitude});
    node["tourism"="hotel"](around:{radius},{latitude},{longitude});
  );
  out body;
"""

# Execute query
result = api.query(query)

# Prepare DataFrame for storing fetched data
data = []
for node in result.nodes:
    lat = node.lat
    lon = node.lon
    name = node.tags.get("name", "No name available")
    amenity = node.tags.get("amenity", "unknown")
    waste_type = (
        "organic" if amenity == "restaurant" else
        "medical" if amenity == "hospital" else
        "mixed" if amenity == "hotel" else
        "unknown"
    )
    data.append({"latitude": lat, "longitude": lon, "name": name, "type": amenity, "waste_type": waste_type})

df = pd.DataFrame(data)

# Save the original data to CSV
original_file = "mansoura_waste_data.csv"
df.to_csv(original_file, index=False)

# Generate additional random data to reach 2000 rows
num_additional_rows = 2000 - len(df)
additional_data = []

for _ in range(num_additional_rows):
    latitude = random.uniform(31.02, 31.05)
    longitude = random.uniform(31.35, 31.37)
    place_type = random.choice(["restaurant", "hospital", "hotel"])
    name = f"Random {place_type.capitalize()} {_}"
    waste_type = "organic" if place_type == "restaurant" else \
                 "medical" if place_type == "hospital" else \
                 "mixed"
    additional_data.append({
        "latitude": latitude,
        "longitude": longitude,
        "name": name,
        "type": place_type,
        "waste_type": waste_type
    })

# Merge original and additional data
additional_df = pd.DataFrame(additional_data)
final_df = pd.concat([df, additional_df], ignore_index=True)

# Save the expanded dataset to CSV
final_file = "expanded_mansoura_waste_data.csv"
final_df.to_csv(final_file, index=False)

# Initialize a map centered on Mansoura
map_combined = folium.Map(location=[latitude, longitude], zoom_start=12)

# Prepare heatmap data
heat_data = final_df[['latitude', 'longitude']].values.tolist()

# Add HeatMap layer to the map
HeatMap(heat_data, radius=25, blur=20).add_to(map_combined)

# Add CircleMarkers for individual locations
for _, row in final_df.iterrows():
    folium.CircleMarker(
        location=[row["latitude"], row["longitude"]],
        radius=7,
        color="blue" if row["type"] == "restaurant" else 
              "red" if row["type"] == "hospital" else 
              "green",
        fill=True,
        fill_color="blue" if row["type"] == "restaurant" else 
                    "red" if row["type"] == "hospital" else 
                    "green",
        fill_opacity=0.6,
        popup=f"Name: {row['name']}<br>Type: {row['type']}<br>Waste: {row['waste_type']}"
    ).add_to(map_combined)

# Save the map as an HTML file
map_file = "waste_heatmap.html"
map_combined.save(map_file)
print(f"Map saved to '{map_file}'")


In [18]:
import folium
from folium.plugins import HeatMap
import pandas as pd

# تحميل بيانات الأماكن الحقيقية من CSV
df = pd.read_csv("mansoura_waste_data.csv")

# إحداثيات مدينة المنصورة (خط العرض والطول)
latitude = 31.0348
longitude = 31.3539

# إنشاء خريطة للمناطق الواقعية فقط
map_combined = folium.Map(location=[latitude, longitude], zoom_start=12)

# إعداد بيانات Heat Map بناءً على نوع النفايات
heat_data = []

# إضافة أماكن على الخريطة مع تأثير حراري
for _, row in df.iterrows():
    # تحديد اللون بناءً على نوع النفايات
    if row["waste_type"] == "organic":
        color = "green"  # نفايات عضوية
        intensity = 0.4  # التأثير الحراري (عند الحاجة لتعديل التأثير)
    elif row["waste_type"] == "medical":
        color = "red"  # نفايات طبية
        intensity = 0.6
    elif row["waste_type"] == "mixed":
        color = "blue"  # نفايات مختلطة
        intensity = 0.5
    else:
        color = "gray"  # غير معروف
        intensity = 0.3

    # إضافة تأثير حراري لكل مكان على الخريطة
    heat_data.append([row["latitude"], row["longitude"], intensity])
    
    # إضافة النقاط على الخريطة
    folium.CircleMarker(
        location=[row["latitude"], row["longitude"]],
        radius=7,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.6,
        popup=f"Name: {row['name']}<br>Type: {row['type']}<br>Waste: {row['waste_type']}"
    ).add_to(map_combined)

# إضافة طبقة HeatMap على الخريطة بناءً على نوع النفايات
HeatMap(
    heat_data,  # إضافة البيانات الحرارية
    radius=25,  # حجم النقاط الحرارية
    blur=15,    # تمويه النقاط لجعلها أكثر نعومة
    gradient={0.2: "green", 0.5: "blue", 0.8: "red", 1: "yellow"}  # تحديد التدرج الحراري
).add_to(map_combined)

# إضافة القدرة على الاختيار بين الطبقات
folium.LayerControl().add_to(map_combined)

# حفظ الخريطة
map_combined.save("heatmap_by_waste_type.html")
print("Map saved to 'heatmap_by_waste_type.html'")


Map saved to 'heatmap_by_waste_type.html'


In [19]:
from IPython.display import IFrame
IFrame('heatmap_by_waste_type.html', width=1000, height=800)


# ##K-Means


In [ ]:
import pandas as pd
import folium
from sklearn.cluster import KMeans
import numpy as np

# تحميل بيانات الأماكن الحقيقية من CSV
df = pd.read_csv("gold.csv")

# تجهيز البيانات للتجميع (Clustering)
# نأخذ الإحداثيات وشدة التأثير (Intensity) أو نوع النفايات إذا أردت
data_for_clustering = df[["latitude", "longitude"]].copy()

# تحويل نوع النفايات إلى أرقام لإضافتها كميزة
waste_type_mapping = {"organic": 1, "medical": 2, "mixed": 3}
df["waste_type_numeric"] = df["waste_type"].map(waste_type_mapping)
data_for_clustering["waste_type_numeric"] = df["waste_type_numeric"]

# تطبيق K-Means Clustering
n_clusters = 4  # عدد المجموعات المطلوبة
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
df["cluster"] = kmeans.fit_predict(data_for_clustering)

# إضافة ألوان لكل Cluster
cluster_colors = {0: "red", 1: "blue", 2: "green", 3: "purple"}

# إنشاء الخريطة
latitude, longitude = 31.0348, 31.3539  # مركز مدينة المنصورة
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# إضافة النقاط إلى الخريطة بناءً على المجموعة (Cluster)
for _, row in df.iterrows():
    folium.CircleMarker(
        location=[row["latitude"], row["longitude"]],
        radius=7,
        color=cluster_colors[row["cluster"]],
        fill=True,
        fill_color=cluster_colors[row["cluster"]],
        fill_opacity=0.6,
        popup=f"Name: {row['name']}<br>Type: {row['type']}<br>Cluster: {row['cluster']}"
    ).add_to(map_clusters)

# حفظ الخريطة
map_clusters.save("clusters_map.html")
print("Map saved to 'clusters_map.html'")
